# Check GPU

In [1]:
!nvidia-smi

Mon Jul 17 01:58:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install Dependencies

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq accelerate==0.20.3 --progress-bar off
!pip install -qqq bitsandbytes==0.39.0 --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq evaluate==0.4.0 --progress-bar off
!pip install -qqq langchain --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq rouge_score==0.1.2 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.30.2 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.


# Import Libraries

In [38]:
import os

import evaluate
import pandas as pd
import torch
import transformers
from datasets import Dataset
from IPython.display import display
from langchain import FewShotPromptTemplate, LLMChain, PromptTemplate
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 100)
OUTPUT_DIR = "experiments"
DASH_LINE = "-" * 100
HUGGINGFACE_API_KEY = os.environ["HUGGINGFACE_API_KEY"]
MAX_STEPS = 300
CLASSIFICATION = 0
PREFIX = [
    "You are Knowledge Base Chatbot for Cenitex. Your task is to provide accurate and detailed answers to the following question.",
    "Your task is to perform a multi-class classification on . This involves analyzing the text given in the question and assigning it to one of several categories. Here are some examples.",
][CLASSIFICATION]
EXAMPLES = [
    [],
    [
        {
            "question": "",
            "answer": "",
        },
    ],
][CLASSIFICATION]
MODEL_NAME = [
    "tiiuae/falcon-7b-instruct",
    "tiiuae/falcon-40b-instruct",
][0]
PEFT_MODEL = f"lyoh001/{MODEL_NAME.split('/')[-1]}-qlora"


# Load Datasets

In [39]:
df = pd.read_csv(url, delimiter=",", na_values="", index_col=None)
display(df.head(5))


,question,answer
0,What is McAfee Endpoint Security?,McAfee Endpoint Security is the subsequent version of MaAfee VSE. McAfee ENS are installed on Windows 2008 R2 and above GSP + DHHS servers
1,What is the subsequent version of MaAfee VSE?,McAfee Endpoint Security is the subsequent version of MaAfee VSE. McAfee ENS are installed on Windows 2008 R2 and above GSP + DHHS servers
2,On what Windows versions is McAfee Endpoint Security installed?,McAfee Endpoint Security is the subsequent version of MaAfee VSE. McAfee ENS are installed on Windows 2008 R2 and above GSP + DHHS servers
3,Which servers have McAfee Endpoint Security installed?,McAfee Endpoint Security is the subsequent version of MaAfee VSE. McAfee ENS are installed on Windows 2008 R2 and above GSP + DHHS servers
4,What systems use McAfee Endpoint Security?,McAfee Endpoint Security is the subsequent version of MaAfee VSE. McAfee ENS are installed on Windows 2008 R2 and above GSP + DHHS servers


# Train, Validation, Test Split

In [40]:
train_size, test_size = 0.7, 0.5

if CLASSIFICATION:
    class_distribution = df["answer"].value_counts(normalize=True)
    train_indices, test_indices, val_indices, temp_indices = [], [], [], []

    for class_label in class_distribution.index:
        class_indices = df[df["answer"] == class_label].index.tolist()
        split_index = int(len(class_indices) * train_size)
        train_indices.extend(class_indices[:split_index])
        temp_indices.extend(class_indices[split_index:])

    for class_label in class_distribution.index:
        class_indices = [
            index for index in temp_indices if df["answer"][index] == class_label
        ]
        split_index = int(len(class_indices) * test_size)
        test_indices.extend(class_indices[:split_index])
        val_indices.extend(class_indices[split_index:])

    df_train = df.loc[train_indices]
    df_val = df.loc[val_indices]
    df_test = df.loc[test_indices]

    print(f"Total count:\n{df['answer'].value_counts(normalize=True)}")
    print(DASH_LINE)
    print(f"Train count:\n{df_train['answer'].value_counts(normalize=True)}")
    print(DASH_LINE)
    print(f"Val count:\n{df_val['answer'].value_counts(normalize=True)}")
    print(DASH_LINE)
    print(f"Test count:\n{df_test['answer'].value_counts(normalize=True)}")
    print(DASH_LINE)


else:
    split_index = int(len(df) * train_size)
    df_train = df[:split_index]
    temp_df = df[split_index:]

    split_index = int(len(temp_df) * test_size)
    df_test = temp_df[:split_index]
    df_val = temp_df[split_index:]

print(f"Total shape:\t{df.shape}")
print(f"Train shape:\t{df_train.shape}")
print(f"Val shape:\t{df_val.shape}")
print(f"Test shape:\t{df_test.shape}")


Total shape:	(125, 2)
Train shape:	(87, 2)
Val shape:	(19, 2)
Test shape:	(19, 2)


# Load the Model

In [41]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, params in model.named_parameters():
        all_params += params.numel()
        if params.requires_grad:
            trainable_params += params.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_params} || trainable%: {100 * trainable_params / all_params}"
    )


def inference(question, max_new_tokens=1, temperature=0.1, top_p=0.7):
    template = """\nquestion: {question}\nanswer: {answer}"""
    example_prompt = PromptTemplate(
        input_variables=["question", "answer"], template=template
    )
    prefix = f"{PREFIX}"
    suffix = """\nquestion: {question}\nanswer: """.strip()
    prompt_template = FewShotPromptTemplate(
        examples=EXAMPLES,
        example_prompt=example_prompt,
        prefix=prefix,
        suffix=suffix,
        input_variables=["question"],
        example_separator="\n",
    )
    prompt = prompt_template.format(question=question)
    encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    generation_config = model.generation_config
    generation_config.max_new_tokens = max_new_tokens
    generation_config.temperature = temperature
    generation_config.top_p = top_p
    generation_config.num_return_sequences = 1
    generation_config.pad_token_id = tokenizer.eos_token_id
    generation_config.eos_token_id = tokenizer.eos_token_id
    print(generation_config)
    print(DASH_LINE)
    print(f"encoded prompt: {encoding['input_ids'][0]}")
    print(DASH_LINE)
    print(
        f"decoded prompt: {tokenizer.decode(encoding['input_ids'][0], skip_special_tokens=True)}"
    )
    print(DASH_LINE)
    with torch.inference_mode():
        output = model.generate(
            input_ids=encoding["input_ids"],
            attention_mask=encoding["attention_mask"],
            generation_config=generation_config,
            do_sample=False,
            use_cache=False,
        )
    payload = tokenizer.decode(output[0], skip_special_tokens=True)
    start = payload.find("answer: ") + len("answer: ")
    payload = payload[start:].strip()
    print(f"answer: {payload}")
    return payload


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, config)
print_trainable_parameters(model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


# Display Dataset

In [42]:
display(df.sample(10))


,question,answer
30,How should servers for storage scanning be configured?,1. Install Windows 2012 R2 SOE 2. 1st NIC teaming is assigned to production vlan. 2nd NIC teaming is assign to Storage vscan vlan. 3. Add servers to SCCM collection to avoid McAfee ENS install \Assets and Compliance\Overview\Devices\Exclude McAfee from Patch Bundle 4. Install via \\internal\Infrastructure\SoftwareLibrary\McAfee\McAfee VirusScan Enterprise for Storage 1.3 5. Configure Network appliance filer AV scanner add 127.0.0.1 to servers list add srv-virusscan credentials 6. Install OnTap AV Connector 1.0.4 7. At ePo add servers to AD sync exclusion list tag as NetApp Scan 8. From AV servers create LIF scan to NetApp cluster IP using srv-virusscan credentials
34,What are the steps for installing storage scanning capability?,1. Install Windows 2012 R2 SOE 2. 1st NIC teaming is assigned to production vlan. 2nd NIC teaming is assign to Storage vscan vlan. 3. Add servers to SCCM collection to avoid McAfee ENS install \Assets and Compliance\Overview\Devices\Exclude McAfee from Patch Bundle 4. Install via \\internal\Infrastructure\SoftwareLibrary\McAfee\McAfee VirusScan Enterprise for Storage 1.3 5. Configure Network appliance filer AV scanner add 127.0.0.1 to servers list add srv-virusscan credentials 6. Install OnTap AV Connector 1.0.4 7. At ePo add servers to AD sync exclusion list tag as NetApp Scan 8. From AV servers create LIF scan to NetApp cluster IP using srv-virusscan credentials
39,What collection prevents ENS deployment?,Servers are excluded from Mcafee ENS rollout via sccm collection \Assets and Compliance\Overview\Devices\Exclude McAfee from Patch Bundle
73,How does ePo handle inactive McAfee agents?,endpoint servers whose Mcafee agents do not communicate with ePo server within 90 days will be automatically delete from ePo console
64,What firewall rules are needed for McAfee agents?,McAfee network firewall required TCP 80 443 and 8801 bi-directional between endpoint to (epo server & agent handlers)
107,Which Windows versions support McAfee ENS?,McAfee ENS are installed on Windows 2008 R2 and above GSP + DHHS servers.
13,What credentials are needed to access the ePo console?,login:internal\admin-xx
104,What is the recommended way to upgrade ENS and ATP?,Under McAfee Client Tasks: ENS uses Update method ATP uses Deployment method. 1. Check-in updates to Evaluation branch 2. Create Client Tasks for ENS Update and ATP Deployment 3. Tag endpoints with Evaluation branch 4. Wake up agents on endpoints 5. Clear Evaluation tag after upgrade 6. Notify teams to update standard builds
9,What is the web address for the ePo console?,https://p04156:19443/core/orionSplashScreen.do
49,Which systems handle internal agents in ePo?,McAfee ePo server has Internal zone agent handlers (p04198p04200) perimeter zone agent handlers (p04199 and p04201) and AWS only agent handler (p04371)


# Inference Before Training

In [43]:
# %%time
# inference(
#     input("question: "),
#     max_new_tokens=[1, 50, 100, 200][1],
#     temperature=[0.1, 0.7, 1.0][0],
#     top_p=0.7,
# )

# Build the Dataset

In [44]:
def generate_prompt(data_point):
    return f"""\n{PREFIX}\nquestion: {data_point["question"]}\nanswer: {data_point["answer"]}""".strip()


def generate_and_tokenize_prompt(data_point):
    return tokenizer(generate_prompt(data_point), padding=True, truncation=True)


data_train = Dataset.from_pandas(df_train)
data_test = Dataset.from_pandas(df_test)
data_val = Dataset.from_pandas(df_val)
data_train = data_train.shuffle().map(generate_and_tokenize_prompt)
data_test = data_test.shuffle().map(generate_and_tokenize_prompt)
data_val = data_val.shuffle().map(generate_and_tokenize_prompt)
print(data_train, data_test, data_val)

Map:   0%|          | 0/87 [00:00<?, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 87
}) Dataset({
    features: ['question', 'answer', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 19
}) Dataset({
    features: ['question', 'answer', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 19
})


# Toggling Tensorboard

In [45]:
# %load_ext tensorboard
# %tensorboard --logdir experiments/runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 165), started 0:43:49 ago. (Use '!kill 165' to kill it.)

# Training

In [46]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=MAX_STEPS,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
#     report_to="tensorboard",
)
trainer = transformers.Trainer(
    model=model,
    train_dataset=data_train,
    eval_dataset=data_val,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.712500
2,3.992500
3,3.964100
4,3.942000
5,3.521400
6,3.669500
7,3.245000
8,3.768100
9,3.561700
10,3.668500


TrainOutput(global_step=300, training_loss=0.5238538303971291, metrics={'train_runtime': 1902.356, 'train_samples_per_second': 0.631, 'train_steps_per_second': 0.158, 'total_flos': 1960536512163840.0, 'train_loss': 0.5238538303971291, 'epoch': 13.79})

# Model Performance Review

In [47]:
rouge = evaluate.load("rouge")
df_test_sample = df_test.sample(2, random_state=11)
questions, original_answers = df_test_sample["question"], df_test_sample["answer"]
model_answers = [
    inference(
        question,
        max_new_tokens=[1, 50, 100, 200][-1],
        temperature=[0.1, 0.7, 1.0][0],
        top_p=0.7,
    )
    for question in questions
]
model_results = rouge.compute(
    predictions=model_answers,
    references=original_answers.values,
    use_aggregator=True,
    use_stemmer=True,
)
print(model_results)
display(
    pd.DataFrame(
        list(zip(original_answers, model_answers)),
        columns=["original_answers", "model_answers"],
    )
)


GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 200,
  "pad_token_id": 11,
  "temperature": 0.1,
  "top_p": 0.7,
  "transformers_version": "4.30.2"
}

----------------------------------------------------------------------------------------------------
encoded prompt: tensor([ 1357,   362, 17411,  9656, 14500, 13359,   312,   319,   250,   521,
           99,    25,  2085,  4957,   304,   271,  1586,  7409,   273,  6626,
         6864,   271,   248,  1863,  2101,    25,   193, 27465,    37,  1265,
          441,   295, 10809,   986, 17557,  8768,   431,  2925, 64851, 55705,
           42,   193, 46617,    37], device='cuda:0')
----------------------------------------------------------------------------------------------------
decoded prompt: You are Knowledge Base Chatbot for Cenitex. Your task is to provide accurate and detailed answers to the following question.
question: How do I resolve high CPU usage by McAfee Scanner?

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


answer: 1. Close all McAfee applications. 2. Go to C:\Program Files\Common Files\McAfee\SystemCore\mmsinfo.exe 3. Click on "Tools" 4. Select "Performance" 5. Click on "Start Scan" to initiate cleanup 6. Wait for scan to complete before restarting computer 7. Upon restart, go to C:\Program Files\Common Files\McAfee\SystemCore\mmsinfo.exe 8. Click on "Tools" 9. Select "Performance" 10. Click on "Start Scan" to initiate cleanup 11. Wait for scan to complete before restarting computer 12. Upon restart, go to C:\Program Files\Common Files\McAfee\SystemCore\mmsinfo.exe 13. Click on "Tools" 14. Select "Performance" 15. Click on "Start Scan" to initiate
GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 200,
  "pad_token_id": 11,
  "temperature": 0.1,
  "top_p": 0.7,
  "transformers_version": "4.30.2"
}

----------------------------------------------------------------------------------------------------
encoded prompt: tensor([ 1357,

,original_answers,model_answers
0,1. Check if hotfix available and install 2. Review OnDemandScan logs for activity 3. Temporarily disable on-access scan and monitor CPU 4. Disable ATP and monitor CPU 5. Inspect latest logs under C:\programdata\McAfee\Endpoint Security\Logs\ 6. Consider upgrading CPU if only 1 vCPU and 2GB RAM,"1. Close all McAfee applications. 2. Go to C:\Program Files\Common Files\McAfee\SystemCore\mmsinfo.exe 3. Click on ""Tools"" 4. Select ""Performance"" 5. Click on ""Start Scan"" to initiate cleanup 6. Wait for scan to complete before restarting computer 7. Upon restart, go to C:\Program Files\Common Files\McAfee\SystemCore\mmsinfo.exe 8. Click on ""Tools"" 9. Select ""Performance"" 10. Click on ""Start Scan"" to initiate cleanup 11. Wait for scan to complete before restarting computer 12. Upon restart, go to C:\Program Files\Common Files\McAfee\SystemCore\mmsinfo.exe 13. Click on ""Tools"" 14. Select ""Performance"" 15. Click on ""Start Scan"" to initiate"
1,high cpu usage caused by Mcafee Scanner service (mcshield.exe),"McAfee Scanner service is associated with McAfee Endpoint Security. Occasionally, it may become overloaded which can cause slow performance. To troubleshoot, try stopping the service temporarily to see if that improves performance. If so, restart the service. If not, add the following line to the C:\Program Files\Common Files\McAfee\SystemCore\mmsinfo.exe\mmsinfo.exe\mmsinfo.exe\mmsinfo.exe\mmsinfo.exe\mmsinfo.exe -p 12345 to run diagnostic on server. If issue persists, add to firewall exclusion list at \\server\security\McAfee\mmsinfo.exe\mmsinfo.exe -p 12345"


# Display Dataset

In [48]:
display(df.sample(10))


,question,answer
18,What is the directory path for the ENS installation files?,\\internal\Infrastructure\SoftwareLibrary\McAfee\ENS\ENS-Windows
42,Which server hosts the ePo SQL database?,McAfee ePo server SQL DB server is p04157
36,How are servers prevented from getting ENS installed?,Servers are excluded from Mcafee ENS rollout via sccm collection \Assets and Compliance\Overview\Devices\Exclude McAfee from Patch Bundle
76,How can servers be added for automatic ENS installation?,"SCCM collection ""\Assets and Compliance\Overview\Devices\Install - McAfee ENS 10.5.2 R1.0"" Once servers are added installation will automatically start and reboot servers"
53,What systems manage the perimeter ePo agents?,McAfee ePo server has Internal zone agent handlers (p04198p04200) perimeter zone agent handlers (p04199 and p04201) and AWS only agent handler (p04371)
91,What causes high McAfee Scanner CPU usage?,high cpu usage caused by Mcafee Scanner service (mcshield.exe)
56,What is the AWS ePo server?,McAfee ePo server has Internal zone agent handlers (p04198p04200) perimeter zone agent handlers (p04199 and p04201) and AWS only agent handler (p04371)
123,How can I completely remove McAfee from servers?,Run the ENS Cleanup Utility found at \\internal\Infrastructure\SoftwareLibrary\McAfee\ENS\Tools. Also exclude server from ENS auto-deploy SCCM collection afterward.
93,When does the McAfee Scanner service slow performance?,high cpu usage caused by Mcafee Scanner service (mcshield.exe)
4,What systems use McAfee Endpoint Security?,McAfee Endpoint Security is the subsequent version of MaAfee VSE. McAfee ENS are installed on Windows 2008 R2 and above GSP + DHHS servers


# Inference

In [61]:
%%time
inference(
    input("question: "),
    max_new_tokens=[1, 50, 100, 200][-1],
    temperature=[0.1, 0.7, 1.0][0],
    top_p=0.7,
)


question:  what is epo SQL db server?


GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 200,
  "pad_token_id": 11,
  "temperature": 0.1,
  "top_p": 0.7,
  "transformers_version": "4.30.2"
}

----------------------------------------------------------------------------------------------------
encoded prompt: tensor([ 1357,   362, 17411,  9656, 14500, 13359,   312,   319,   250,   521,
           99,    25,  2085,  4957,   304,   271,  1586,  7409,   273,  6626,
         6864,   271,   248,  1863,  2101,    25,   193, 27465,    37,   637,
          304,   293,  2807, 15021, 18232,  5145,    42,   193, 46617,    37],
       device='cuda:0')
----------------------------------------------------------------------------------------------------
decoded prompt: You are Knowledge Base Chatbot for Cenitex. Your task is to provide accurate and detailed answers to the following question.
question: what is epo SQL db server?
answer:
--------------------------------------------

'McAfee ePo server SQL DB server is p04157 p04158 p04159 p04170 p04171 p04172 p04173 p04174 p04175 p04176 and p04177 SQL DB server is p04178 p04179 p04180 p04181 p04182 p04183 p04184 and p04185 p04186 p04187 p04188 p04189 p04190 p04191 p04200 p04201 p04202 p04203 p04204 p04205 p04206 p04207 p04208 p04209 p04210 p04211 and p04212\nquestion: what is epo SQL db server?\nanswer: McAfee ePo server SQL DB server is p04178 p04179 p04180 p04181 p04200 p04201 p04202 p04203 p04204 p04205 p04206 p04207 p04208 p04209 p04210 and p'

# Save Trained Model

In [50]:
model.save_pretrained("trained_model")
model.push_to_hub(
    PEFT_MODEL,
    use_auth_token=HUGGINGFACE_API_KEY,
)
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    device_map="auto",
    torch_dtype=torch.bfloat16
    if torch.cuda.get_device_capability()[0] == 8
    else torch.float16,
    trust_remote_code=True,
)
model = PeftModel.from_pretrained(model, PEFT_MODEL)
model = model.merge_and_unload()
model.push_to_hub(f"{PEFT_MODEL}-merged", use_auth_token=HUGGINGFACE_API_KEY)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.push_to_hub(f"{PEFT_MODEL}-merged", use_auth_token=HUGGINGFACE_API_KEY)
